# Lab 08 Exercises: Linear Algebra (PCA)

## 1\. PCA on 3D dataset

1. Generate a dataset with $M = 3$ features and N ${\cal O}(1000)$ measurements. Specifically, with $N(\mu,\sigma)$ the normal distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 features $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
2. Find the eigenvectors and eigenvalues of the covariance matrix of the dataset.
3. Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result, being the covariance matrix symmetric.
4. What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
5. Redefine the data in the basis yielded by the PCA procedure.
6. Plot the data points in the original and the new coordinates as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


### 1.

Generate a dataset with $M = 3$ features and N ${\cal O}(1000)$ measurements. Specifically, with $N(\mu,\sigma)$ the normal distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 features $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$

In [1]:
import pandas as pd
from numpy import random
import numpy as np
random.seed(102340)

N = 1000
x1 = random.normal(loc= 0, scale = 1, size = N)
x2 = x1 + random.normal(loc= 0, scale = 3, size = N)
x3 = 2*x1 + x2

df = pd.DataFrame(
    {
        'x1': x1,
        'x2': x2,
        'x3': x3
    }
)
df

,x1,x2,x3
0,0.626709,0.104184,1.357602
1,0.984555,-7.457642,-5.488532
2,-0.435850,-2.371178,-3.242878
3,-0.347136,5.250045,4.555774
4,2.192992,4.451389,8.837374
...,...,...,...
995,0.599057,-1.216198,-0.018083
996,-1.974952,-2.296643,-6.246547
997,0.217803,-0.170506,0.265100
998,0.229704,0.964773,1.424180


### 2.

Find the eigenvectors and eigenvalues of the covariance matrix of the dataset.

In [2]:
# Covariance

# To feed the np.cov built in function, the data shall be passed as an array with nrows= features, ncols= N
data_array = np.vstack((x1, x2, x3))
print(data_array.shape)
covariance_matrix = np.cov(data_array)

covariance_matrix

# Eigenvalues, eigenvectors using diagonalization
from scipy import linalg as la
eigval, eigvect = la.eig(covariance_matrix)

print ("eigenvectors: \n", eigvect, end = '\n\n')
print ("eigenvalues: \n", eigval, end = '\n\n') # by default, this returns complex values. but in this case (cov is symmetric)...
print("eigenvalues (real): \n", np.real_if_close(eigval)) # ... we now the eigval are real

(3, 1000)
eigenvectors: 
 [[-0.11185642 -0.81649658  0.56641105]
 [-0.5818526  -0.40824829 -0.70340663]
 [-0.80556545  0.40824829  0.42941547]]

eigenvalues: 
 [2.61765766e+01+0.j 2.17820035e-16+0.j 2.07652988e+00+0.j]

eigenvalues (real): 
 [2.61765766e+01 2.17820035e-16 2.07652988e+00]


### 3. 

Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result, being the covariance matrix symmetric.

In [3]:
# Eigenvalues, eigenvectors using SVD
# For a square matrix, these must be the same

U, spectrum, Vt = la.svd(covariance_matrix)

print (spectrum,'\n\n')
print (U,'\n\n')
#print (Vt,'\n\n')
print( Vt.T, '\n\n') # U and V are the same, as expected for a square matrix
# the eigenvalue order has changed because the vector space basis has changed too (see component 2 and 3 of each eigvect is inverted)

[2.61765766e+01 2.07652988e+00 1.06804701e-15] 


[[-0.11185642  0.56641105 -0.81649658]
 [-0.5818526  -0.70340663 -0.40824829]
 [-0.80556545  0.42941547  0.40824829]] 


[[-0.11185642  0.56641105 -0.81649658]
 [-0.5818526  -0.70340663 -0.40824829]
 [-0.80556545  0.42941547  0.40824829]] 





As we would have expected, there is an almost zero eigenvalue: this corresponds to the fact that x3 is a linear combination of x1 and x2 with no statistical noise at all.

The two other eigenvalues have magnitude ratio 1:10. It corresponds to the fact that the std of the two variables x1 and x2 is different.


### 4. 

What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.

In [4]:
eigval = np.real_if_close(eigval)
Lambda_ = np.diag(eigval)
Lambda_

p = 1 # principal components retained
e
explained_variability = np.sum

array([[2.61765766e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.17820035e-16, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.07652988e+00]])

In [5]:
# Rotate the data according to the new basis

covariance_matrix_rotated = eigvect.T @ covariance_matrix @ eigvect

np.zero

AttributeError: module 'numpy' has no attribute 'zero'

## 2\. PCA on a nD dataset

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

## 3 \. Looking at an oscillating spring (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


## 4\. PCA on the MAGIC dataset (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 